In [ ]:
import pandas as pd
import numpy as np
import requests

In [ ]:
df = pd.read_csv('final_data.csv')

In [ ]:
df = df.drop('Unnamed: 0',axis = 1)

In [ ]:
df.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...


In [ ]:
df.isnull().sum()

director_name      0
actor_1_name       0
actor_2_name      26
actor_3_name     109
genres             6
movie_title        0
dtype: int64

In [ ]:
df.fillna('unknown',inplace=True)

In [ ]:

(df["director_name"][1])
print(''.join(df["director_name"][1]).replace(' ',''))

GoreVerbinski


In [ ]:
def remove_space(x):
  temp = (''.join(x).replace(' ',''))
  return temp

In [ ]:
df['director_name'] = df['director_name'].apply(remove_space)
df['actor_1_name'] = df['actor_1_name'].apply(remove_space)
df['actor_2_name'] = df['actor_2_name'].apply(remove_space)
df['actor_3_name'] = df['actor_3_name'].apply(remove_space)


In [ ]:
df.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,JamesCameron,CCHPounder,JoelDavidMoore,WesStudi,Action Adventure Fantasy Sci-Fi,avatar
1,GoreVerbinski,JohnnyDepp,OrlandoBloom,JackDavenport,Action Adventure Fantasy,pirates of the caribbean: at world's end
2,SamMendes,ChristophWaltz,RoryKinnear,StephanieSigman,Action Adventure Thriller,spectre
3,ChristopherNolan,TomHardy,ChristianBale,JosephGordon-Levitt,Action Thriller,the dark knight rises
4,DougWalker,DougWalker,RobWalker,unknown,Documentary,star wars: episode vii - the force awakens ...


In [ ]:
type(df['genres'][0])

str

In [ ]:
df['tags']= df['director_name']+' '+df['actor_1_name']+' '+df['actor_2_name']+' '+df['actor_3_name']+' '+df['genres']

In [ ]:
#!pip install tmdbv3api

In [ ]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '8265bd1679663a7ea12ac168da84d2e8'

In [ ]:
# Get id's of the movies
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_id(x):
  result = tmdb_movie.search(x)
  k = []
  movie_id=0
  for i in result:
    k.append(i['id'])
    if len(k) != 0:
      movie_id = k[0]
  return movie_id
        

In [ ]:
get_id('avatar')

19995

In [ ]:
df['movie_id'] = df['movie_title'].map(lambda x: get_id(x))

In [ ]:
df[df['movie_id']==19995]

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,tags,movie_id
0,JamesCameron,CCHPounder,JoelDavidMoore,WesStudi,Action Adventure Fantasy Sci-Fi,avatar,JamesCameron CCHPounder JoelDavidMoore WesStud...,19995


In [ ]:
# Get overview of movies from tmdb api
def get_overview(movie_id):
    overview = " "
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if 'overview' in data_json:
          overview=(data_json.get('overview'))
          return overview
    else:
       return np.NaN

In [ ]:
get_overview1(19995)

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [ ]:
df['movie_overview'] = df['movie_id'].map(lambda x: get_overview(x))

In [ ]:
df.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,tags,movie_id,movie_overview
0,JamesCameron,CCHPounder,JoelDavidMoore,WesStudi,Action Adventure Fantasy Sci-Fi,avatar,JamesCameron CCHPounder JoelDavidMoore WesStud...,19995,"In the 22nd century, a paraplegic Marine is di..."
1,GoreVerbinski,JohnnyDepp,OrlandoBloom,JackDavenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,GoreVerbinski JohnnyDepp OrlandoBloom JackDave...,285,"Captain Barbossa, long believed to be dead, ha..."
2,SamMendes,ChristophWaltz,RoryKinnear,StephanieSigman,Action Adventure Thriller,spectre,SamMendes ChristophWaltz RoryKinnear Stephanie...,206647,A cryptic message from Bond’s past sends him o...
3,ChristopherNolan,TomHardy,ChristianBale,JosephGordon-Levitt,Action Thriller,the dark knight rises,ChristopherNolan TomHardy ChristianBale Joseph...,49026,Following the death of District Attorney Harve...
4,DougWalker,DougWalker,RobWalker,unknown,Documentary,star wars: episode vii - the force awakens ...,DougWalker DougWalker RobWalker unknown Docume...,140607,Thirty years after defeating the Galactic Empi...


## Remove stop words and stemming




In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
def tokenize(text):
    word_tokens = word_tokenize(str(text))
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    tokens_str = " "
    return tokens_str.join(filtered_sentence)

In [ ]:
df['movie_overview'] = df['movie_overview'].apply(tokenize)

In [ ]:
new_df = df[['movie_id','movie_title','tags','movie_overview']]

## Stemming 

In [ ]:
import nltk 
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
  tokenAux=""
  textAux=""
  tokens = nltk.word_tokenize(text)
  for token in tokens:
      tokenAux = token
      tokenAux = ps.stem(token)    
      textAux = textAux + " "+ tokenAux
  return textAux

In [ ]:
new_df['movie_overview']=new_df['movie_overview'].apply(stem)

In [ ]:
new_df['tags'] = (new_df['tags']+' '+new_df['movie_overview']).str.lower()

In [ ]:
new_df.drop('movie_overview',axis=1,inplace = True)

## Converting to vectors

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=7000,stop_words='english')

In [ ]:
vector = cv.fit_transform(new_df['tags'])

In [ ]:
cv.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
new_df[new_df['movie_title'] == 'batman']

,movie_id,movie_title,tags
1400,414906,batman,timburton michaelgough jackpalance williamhoot...


In [ ]:
sorted(list(enumerate(similarity[1400])),reverse = True,key = lambda x : x[1])[1:6] # Lambda is used to sortfrom index 1 and not 0

[(6226, 0.7460038465922509),
 (4457, 0.34641016151377546),
 (441, 0.31008683647302115),
 (120, 0.28112676511587464),
 (784, 0.28112676511587464)]

In [ ]:
new_df.iloc[4407][1]

'kissing jessica stein'

In [ ]:
new_df.shape

(6398, 3)

## Recommend function

In [ ]:
def recommend(movie):
    index = new_df[new_df['movie_title'] == movie].index[0]
    distances = similarity[index]
    movies_list = sorted(list(enumerate(distances)),reverse = True,key = lambda x : x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].movie_title)

        

In [ ]:
recommend('gandhi')

gandhi, my father
a passage to india
guiana 1838
dil jo bhi kahey...
mr. turner


In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))